In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 215kB 2.8MB/s 
     |████████████████████████████████| 1.7MB 8.9MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
     |████████████████████████████████| 317kB 21.7MB/s 
     |████████████████████████████████| 7.4MB 26.5MB/s 
     |████████████████████████████████| 2.9MB 47.4MB/s 
     |████████████████████████████████| 1.1MB 44.4MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 122kB 51.3MB/s 
     |████████████████████████████████| 163kB 53.1MB/s 
     |████████████████████████████████| 102kB 11.6MB/s 
     |████████████████████████████████| 102kB 11.4MB/s 
     |████████████████████████████████| 6.7MB 40.9MB/s 
     |████████████████████████████████| 4.4MB 46.7MB/s 
     |████████████████████████████████| 133kB 46.3MB/s 
     |████████████████████████████████| 112kB 52.6MB/s 
     |████████████████████████████████| 1.1MB 37.3MB/s 
     |████████████████████████████████| 890kB 43.5MB/s

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import *
from sklearn.model_selection import *
from scipy.special import softmax
import re
from sklearn.model_selection import StratifiedKFold
from simpletransformers.classification import MultiLabelClassificationModel
from simpletransformers.classification.classification_model import ClassificationModel
from sklearn.metrics import mean_squared_error as mse
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt
from sklearn.utils import class_weight

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
df_train = pd.read_json("/content/drive/My Drive/Embold_Participant's_Dataset/Embold_Participant's_Dataset/embold_train.json")
df_test = pd.read_json("/content/drive/My Drive/Embold_Participant's_Dataset/Embold_Participant's_Dataset/embold_test.json")
df_extra_train = pd.read_json("/content/drive/My Drive/Embold_Participant's_Dataset/Embold_Participant's_Dataset/embold_train_extra.json")

In [ ]:
df_train.head()

,title,body,label
0,y-zoom piano roll,a y-zoom on the piano roll would be useful.,1
1,buggy behavior in selection,! screenshot from 2016-02-23 21 27 40 https:/...,0
2,auto update feature,"hi,\r \r great job so far, @saenzramiro ! : \r...",1
3,filter out noisy endpoints in logs,i think we should stop logging requests to:\r ...,1
4,enable pid on / pid off alarm actions for ardu...,expected behavior\r alarm actions pid on and p...,0


In [ ]:
df_test.shape

(30000, 2)

In [ ]:
df_train.shape

(150000, 3)

In [ ]:
df_extra_train.shape

(300000, 3)

In [ ]:
df_train['label'].value_counts(normalize=True)

1    0.460707
0    0.445513
2    0.093780
Name: label, dtype: float64

In [ ]:
df_extra_train['label'].value_counts(normalize=True)

1    0.460707
0    0.445513
2    0.093780
Name: label, dtype: float64

In [ ]:
df_temp = pd.concat([df_train,df_extra_train,df_test])
df_temp['final_text'] = df_temp['title']
# df_temp['body'] = df_temp['body'].apply(lambda x:' '.join(re.sub(r"[^A-Za-z]"," ",x).split()))
df_temp['len'] = df_temp['final_text'].apply(lambda x: len(x.split()))
df_temp['len'].describe()

count    480000.000000
mean          6.952004
std           3.271260
min           2.000000
25%           5.000000
50%           6.000000
75%           9.000000
max          50.000000
Name: len, dtype: float64

In [ ]:
df_temp = df_temp[df_temp['len'] != 0] 

In [ ]:
df_train = df_temp[df_temp['label'].notnull()]
df_test = df_temp[df_temp['label'].isnull()]

In [ ]:
X_train = pd.DataFrame()
X_train['text'] = df_train['final_text']
X_train['labels'] = df_train['label']

In [ ]:
test_temp = pd.DataFrame()
test_temp['text'] = df_test['final_text']
test_temp['labels'] = 0

In [ ]:
model = ClassificationModel('bert','allenai/scibert_scivocab_uncased', num_labels=3,args={'train_batch_size':32, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 2, 'max_seq_length': 100,'manual_seed': 2,'overwrite_output_dir': True,'verbose':False})

KeyboardInterrupt: ignored

In [ ]:
X_train.shape

(450000, 2)

In [ ]:
n_folds=5
folds=StratifiedKFold(n_splits=n_folds,shuffle=True,random_state=22)
avg_cv = []

import pickle
final_preds =np.zeros([len(df_test),3]) 
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train.values,X_train['labels'].values)):
        print("Fold {}".format(fold_))
        X_trn = X_train.iloc[trn_idx]
        X_val = X_train.iloc[val_idx]
        # preds_lgb = clf.predict_proba(X_val)
        # with open(f"/content/drive/My Drive/Embold_Participant's_Dataset/Embold_Participant's_Dataset/val_idx_fold_5F_scibert_{fold_}.pkl",'wb') as f:
        #    pickle.dump(val_idx,f)
        model = ClassificationModel('bert','allenai/scibert_scivocab_uncased', num_labels=3,args={'train_batch_size':64, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 2, 'max_seq_length': 12,'manual_seed': 2,'overwrite_output_dir': True})
        model.train_model(X_trn)
        result, model_outputs, wrong_predictions = model.eval_model(X_val)
        model_outputs = softmax(model_outputs,axis=1)
        with open(f"/content/drive/My Drive/Embold_Participant's_Dataset/Embold_Participant's_Dataset/preds_xlnet_base_title{fold_}.pkl",'wb') as f:
          pickle.dump(model_outputs,f)
        preds = [np.argmax(x) for x in model_outputs]
        result, model_outputs, wrong_predictions = model.eval_model(test_temp)
        model_outputs = softmax(model_outputs,axis=1)
        avg_cv.append(accuracy_score(y_pred=preds,y_true=X_val['labels']))
        final_preds += model_outputs
        # final_preds += clf.predict_proba(df_test)

final_preds = final_preds/n_folds
with open(f"/content/drive/My Drive/Embold_Participant's_Dataset/Embold_Participant's_Dataset/final_preds_xlnet_base_title_fin.pkl",'wb') as f:
           pickle.dump(final_preds,f)
print(sum(avg_cv)/n_folds)

Fold 0


Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)



Fold 1


...


Fold 3


Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we


Fold 4


Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification we


0.7941311111111112


In [ ]:
import pickle
with open(f"/content/drive/My Drive/Embold_Participant's_Dataset/Embold_Participant's_Dataset/final_preds_scibert_full.pkl",'rb') as f:
           final_preds = pickle.load(f)

In [ ]:
# softmax(model_outputs,axis=1)

In [ ]:
model.train_model(train_temp)

NameError: ignored

In [ ]:
# test_temp

In [ ]:
test_temp = pd.DataFrame()
test_temp['text'] = df_test['body']
test_temp['labels'] = 0

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test_temp)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [ ]:
preds = [np.argmax(x) for x in final_preds]

In [ ]:
df_sub = pd.read_csv("/content/drive/My Drive/Embold_Participant's_Dataset/Embold_Participant's_Dataset/sample submission.csv")

In [ ]:
df_sub['label'] = preds

In [ ]:
df_sub['label'].value_counts(normalize=True)

1    0.469400
0    0.461833
2    0.068767
Name: label, dtype: float64

In [ ]:
df_sub.to_csv('Exp_baseline_body_title_2ep_scibert_full.csv',index=False)

In [ ]:
2 Ep 5fold 
83.667